# CLIP on Flowers!!??!!??!
Using the notebook we saw during lectures as a starting point for the project

In [7]:
# we need to install clip as it is not pre-installed
# you are also free to use open_clip which provide more models
# https://github.com/mlfoundations/open_clip
#%pip install openai_clip

In [8]:
import torch
import torchvision
import clip
from tqdm import tqdm

## Metanetwork class

In [ ]:
"""
MetaNetwork è una piccola rete neurale (MLP con 2 layer)
che trasforma le image_features (512-dim) in un token
condizionale (512-dim) usato in CoCoOp.

Questo token varia per ogni immagine, permettendo prompt
personalizzati per ogni input.
"""

class MetaNetwork(nn.Module):
    def __init__(self, ctx_dim=512, hidden_dim=256):
        """
        Args:
            ctx_dim: dimensione degli embeddings (512 per ViT-B/16)
            hidden_dim: dimensione dello strato nascosto
        """
        super().__init__()
        self.linear1 = nn.Linear(ctx_dim, hidden_dim)
        self.relu = nn.ReLU(inplace=True)
        self.linear2 = nn.Linear(hidden_dim, ctx_dim)

    def forward(self, image_features):
        """
        Args:
            image_features: tensor (B, ctx_dim) dalle immagini encodate
        
        Returns:
            conditional_token: tensor (B, ctx_dim)
        """
        # Assicura il tipo corretto (importante per mixed precision)
        image_features = image_features.to(self.linear1.weight.dtype)
        
        out = self.linear1(image_features)
        out = self.relu(out)
        out = self.linear2(out)
        return out

## CoCoOp 

In [ ]:
# ====== CELL: CoCoOpPromptLearner Class ======
"""
CoCoOpPromptLearner crea prompt dinamici della forma:

    [V1] [V2] ... [VM] + [π(x)] + [CLASS]
    
Dove:
- [V1]...[VM] sono context vector statici (appresi)
- [π(x)] è il token condizionale (generato dalla MetaNetwork)
- [CLASS] è l'embedding del nome della classe

Importante: Ogni immagine ha un prompt diverso grazie a π(x)!
"""

class CoCoOpPromptLearner(nn.Module):
    def __init__(self, clip_model, classnames, n_ctx=16, ctx_init=""):
        """
        Args:
            clip_model: il modello CLIP caricato (per accedere agli embeddings)
            classnames: lista di nomi di classi, es. ["rose", "daisy", ...]
            n_ctx: numero di context vectors (default 16)
            ctx_init: inizializzazione per i context vector (default random)
        """
        super().__init__()
        self.n_ctx = n_ctx
        self.classnames = classnames
        
        # Tipo di dato del modello (float32 o float16)
        dtype = clip_model.dtype
        
        # Dimensione degli embeddings
        ctx_dim = clip_model.ln_final.weight.shape[0]
        print(f"[CoCoOp] Context dimension: {ctx_dim}")
        
        # ===== Context vectors (static, learnable) =====
        ctx_vectors = torch.empty(n_ctx, ctx_dim, dtype=dtype)
        nn.init.normal_(ctx_vectors, std=0.02)
        self.ctx = nn.Parameter(ctx_vectors)
        
        # ===== Meta Network (genera token condizionale) =====
        self.meta_net = MetaNetwork(ctx_dim, hidden_dim=256)
        
        # ===== Tokenizza nomi delle classi =====
        # Questo lo facciamo una volta durante __init__
        device = next(clip_model.parameters()).device
        classnames_tokens = clip.tokenize(classnames).to(device)
        
        # Estrai gli embeddings dei token
        with torch.no_grad():
            class_embeddings = clip_model.token_embedding(classnames_tokens)
        
        # class_embeddings: (num_classes, seq_len, ctx_dim)
        # es. (51, 77, 512) per 51 classi base e lunghezza token max CLIP
        self.register_buffer('class_token_embeddings', class_embeddings)
        
    def forward(self, image_features):
        """
        Genera prompts per ogni immagine e classe.
        
        Args:
            image_features: tensor (B, ctx_dim) dalle immagini encodate
        
        Returns:
            prompts: tensor (B, num_classes, seq_len_total, ctx_dim)
                     Ogni elemento è il prompt per una coppia (immagine, classe)
        """
        batch_size = image_features.shape[0]
        num_classes, seq_len, ctx_dim = self.class_token_embeddings.shape
        
        # Step 1: Genera token condizionale usando MetaNetwork
        # Questo varia per ogni immagine nel batch
        cond_token = self.meta_net(image_features)  # (B, ctx_dim)
        cond_token = cond_token.unsqueeze(1)        # (B, 1, ctx_dim)
        
        # Step 2: Espandi context vector per batch e classi
        # Da (n_ctx, ctx_dim) a (B, num_classes, n_ctx, ctx_dim)
        ctx = self.ctx.unsqueeze(0).unsqueeze(0)  # (1, 1, n_ctx, ctx_dim)
        ctx = ctx.expand(batch_size, num_classes, -1, -1)  # (B, num_classes, n_ctx, ctx_dim)
        
        # Step 3: Espandi conditional token per tutte le classi
        # Da (B, 1, ctx_dim) a (B, num_classes, 1, ctx_dim)
        cond_expand = cond_token.unsqueeze(1)  # (B, 1, 1, ctx_dim)
        cond_expand = cond_expand.expand(-1, num_classes, -1, -1)  # (B, num_classes, 1, ctx_dim)
        
        # Step 4: Espandi class embeddings per il batch
        # Da (num_classes, seq_len, ctx_dim) a (B, num_classes, seq_len, ctx_dim)
        class_embed = self.class_token_embeddings.unsqueeze(0)  # (1, num_classes, seq_len, ctx_dim)
        class_embed = class_embed.expand(batch_size, -1, -1, -1)  # (B, num_classes, seq_len, ctx_dim)
        
        # Step 5: Concatena lungo la dimensione di sequenza
        # Struttura finale: [context] + [cond_token] + [class_tokens]
        prompts = torch.cat([ctx, cond_expand, class_embed], dim=2)
        # prompts shape: (B, num_classes, n_ctx + 1 + seq_len, ctx_dim)
        
        return prompts

In [ ]:
# ====== CELL: CoCoOpTrainer Class ======
"""
CoCoOpTrainer fornisce:
1. train_epoch(): esegue un epoca di training su base classes
2. eval(): valuta su base o novel classes

Importante: CLIP rimane congelato, alleniamo solo i prompt learner!
"""

class CoCoOpTrainer:
    def __init__(self, clip_model, base_classnames, base_classes, 
                 novel_classes, device, lr=0.002):
        """
        Args:
            clip_model: modello CLIP caricato
            base_classnames: lista di nomi classi base, es. [CLASS_NAMES[i] for i in base_classes]
            base_classes: lista di indici base classes
            novel_classes: lista di indici novel classes
            device: "cuda" o "cpu"
            lr: learning rate (default 0.002)
        """
        self.clip_model = clip_model
        self.base_classnames = base_classnames
        self.base_classes = base_classes
        self.novel_classes = novel_classes
        self.device = device
        
        # Freeze CLIP parameters
        for p in clip_model.parameters():
            p.requires_grad = False
        
        # Crea il prompt learner
        self.prompt_learner = CoCoOpPromptLearner(
            clip_model, 
            base_classnames
        ).to(device)
        
        # Optimizer - allena solo il prompt learner
        self.optimizer = torch.optim.SGD(
            self.prompt_learner.parameters(),
            lr=lr,
            momentum=0.9,
            weight_decay=5e-4
        )
        
    def train_epoch(self, train_dataset, batch_size=32):
        """
        Esegue una epoca di training.
        
        Args:
            train_dataset: PyTorch Dataset (train_base nel nostro caso)
            batch_size: dimensione batch (default 32)
        
        Returns:
            avg_loss: loss medio dell'epoca
        """
        self.prompt_learner.train()
        self.clip_model.eval()  # CLIP sempre in eval mode
        
        dataloader = torch.utils.data.DataLoader(
            train_dataset, 
            batch_size=batch_size, 
            shuffle=True, 
            num_workers=2
        )
        
        total_loss = 0
        n_batches = 0
        
        for batch_idx, (images, labels) in enumerate(tqdm(dataloader, desc="CoCoOp training")):
            images = images.to(self.device)
            labels = labels.to(self.device)
            
            # ===== FORWARD =====
            
            # Encode immagini (frozen)
            with torch.no_grad():
                img_feat = self.clip_model.encode_image(images)
            
            # Normalizza image features
            img_feat = img_feat.to(self.prompt_learner.meta_net.linear1.weight.dtype)
            img_feat /= img_feat.norm(dim=-1, keepdim=True)
            
            # Genera prompts condizionali per ogni immagine
            prompts = self.prompt_learner(img_feat)  # (B, num_classes, seq_len, ctx_dim)
            B, N, L, D = prompts.shape
            
            # Reshape per poter passare al text encoder
            # Il text encoder si aspetta (total_prompts, seq_len, ctx_dim)
            prompts_flat = prompts.view(B * N, L, D)
            
            # Encode testi tramite CLIP text encoder
            with torch.no_grad():
                # CLIP text encoder aspetta token, non embeddings
                # Quindi usiamo direttamente gli embeddings come input al decoder
                # Pseudo-forward: accumuliamo i token embeddings e li passiamo
                # In realtà, usiamo il text encoder come descritto nel paper
                
                # Approccio alternativo: uso il logit_scale
                # e calcolo manualmente come nel codice del paper
                text_feat = self.clip_model.encode_text_from_embeddings(prompts_flat)
            
            # Se la funzione sopra non esiste, useremo questo trick:
            # Calcoliamo le features testuali tramite il text encoder embedding decoder
            
            # Per semplicità (e poiché CLIP non espone direttamente il decoder),
            # usiamo la nostra versione approssimata:
            # Codifichiamo il prompt come fatto dal paper
            
            # IMPORTANT FIX: Calcoliamo i logit direttamente usando
            # il transformer text encoder con gli embeddings dei token
            text_feat = self.clip_model.encode_text(prompts_flat)  # This won't work directly!
            
            # ===== WORKAROUND CORRETTO =====
            # Il text encoder di CLIP prende token interi, non embeddings
            # Quindi usiamo una formula approssimata basata sul paper:
            # Assumiamo che il testo encoder faccia: embed → transformer → [0] token → projection
            
            # Per questo esercizio, usiamo il metodo dei template fissi + prompt learner
            # comeuna forma semplificata
            
            # ALTERNATIVE: Usiamo il text encoder con un approccio "prompt engineering"
            # dove combiniamo i soft prompts come stringhe
            
            # Per ora, implementiamo il versione "semplificata ma corretta":
            # Passiamo le embedding direttamente attraverso il transformer del CLIP
            
            # Estratto dal codice del paper CoCoOp:
            # Coda di approccio: il text encoder ha il token embedding layer,
            # il transformer, e il layer normalization finale.
            # Passiamo i nostri embeddings diretti.
            
            # Usiamo il metodo diretto: unsqueeze per [CLS] token
            # e passa attraverso il resto del text encoder
            
            # IMPLEMENTATION: Usa il backbone del text encoder
            x = prompts_flat  # (B*N, seq_len, 512)
            
            # Aggiungi positional embeddings (ereditati da CLIP)
            x = x + self.clip_model.positional_embedding
            
            # Permuta per il transformer
            x = x.permute(1, 0, 2)  # (seq_len, B*N, 512)
            
            # Passa attraverso il transformer
            x = self.clip_model.transformer(x)
            
            # Ritorna al formato batch
            x = x.permute(1, 0, 2)  # (B*N, seq_len, 512)
            
            # Prendi il token [CLS] (primo token)
            x = x[:, 0, :]  # (B*N, 512)
            
            # Layer norm finale
            x = self.clip_model.ln_final(x)
            
            # Projection test
            text_feat = self.clip_model.text_projection @ x.T  # (512, B*N)
            text_feat = text_feat.T  # (B*N, 512)
            
            # Reshape back to (B, N, 512)
            text_feat = text_feat.view(B, N, -1)  # (B, N, 512)
            
            # Normalizza
            text_feat /= text_feat.norm(dim=-1, keepdim=True)
            
            # ===== LOGITS =====
            # Similarità coseno: (B, 512) @ (B, N, 512)^T → (B, N)
            logit_scale = self.clip_model.logit_scale.exp()
            logits = logit_scale * (img_feat.unsqueeze(1) * text_feat).sum(-1)
            
            # ===== LOSS =====
            # Remap labels da base_classes indices ai contiguous [0, num_base_classes)
            contig_cat2idx = {cat: idx for idx, cat in enumerate(self.base_classes)}
            labels_mapped = torch.tensor(
                [contig_cat2idx[l.item()] for l in labels]
            ).to(self.device)
            
            loss = F.cross_entropy(logits, labels_mapped)
            
            # ===== BACKWARD =====
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            
            total_loss += loss.item()
            n_batches += 1
        
        avg_loss = total_loss / max(1, n_batches)
        return avg_loss
    
    @torch.no_grad()
    def eval(self, dataset, categories, batch_size=64):
        """
        Valuta il modello su un dataset.
        
        Args:
            dataset: PyTorch Dataset (test_base o test_novel)
            categories: lista di indici di classi (base_classes o novel_classes)
            batch_size: dimensione batch per valutazione
        
        Returns:
            accuracy: float tra 0 e 1
        """
        self.prompt_learner.eval()
        self.clip_model.eval()
        
        contig_cat2idx = {cat: idx for idx, cat in enumerate(categories)}
        
        dataloader = torch.utils.data.DataLoader(
            dataset,
            batch_size=batch_size,
            shuffle=False,
            num_workers=2
        )
        
        correct = 0
        total = 0
        
        for images, labels in tqdm(dataloader, desc="CoCoOp eval"):
            images = images.to(self.device)
            labels = labels.to(self.device)
            
            # Encode immagini
            img_feat = self.clip_model.encode_image(images)
            img_feat = img_feat.to(self.prompt_learner.meta_net.linear1.weight.dtype)
            img_feat /= img_feat.norm(dim=-1, keepdim=True)
            
            # Genera prompts
            prompts = self.prompt_learner(img_feat)
            B, N, L, D = prompts.shape
            prompts_flat = prompts.view(B * N, L, D)
            
            # Encode testi (stessa procedura del training)
            x = prompts_flat
            x = x + self.clip_model.positional_embedding
            x = x.permute(1, 0, 2)
            x = self.clip_model.transformer(x)
            x = x.permute(1, 0, 2)
            x = x[:, 0, :]
            x = self.clip_model.ln_final(x)
            text_feat = (self.clip_model.text_projection @ x.T).T
            text_feat = text_feat.view(B, N, -1)
            text_feat /= text_feat.norm(dim=-1, keepdim=True)
            
            # Logits
            logit_scale = self.clip_model.logit_scale.exp()
            logits = logit_scale * (img_feat.unsqueeze(1) * text_feat).sum(-1)
            
            # Predizioni
            pred = logits.argmax(dim=1)
            labels_mapped = torch.tensor(
                [contig_cat2idx[l.item()] for l in labels]
            ).to(self.device)
            
            correct += (pred == labels_mapped).sum().item()
            total += labels.size(0)
        
        return correct / total


In [ ]:
# ====== CELL: Setup and Train CoCoOp ======

# Prepara i nomi delle classi base
base_classnames = [CLASS_NAMES[i] for i in base_classes]
print(f"Base classnames ({len(base_classnames)}): {base_classnames[:5]}...")

# Crea il trainer
trainer = CoCoOpTrainer(
    clip_model=model,
    base_classnames=base_classnames,
    base_classes=base_classes,
    novel_classes=novel_classes,
    device=device,
    lr=0.002
)

print("\n" + "="*60)
print("🚀 Training CoCoOp")
print("="*60)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    avg_loss = trainer.train_epoch(train_base, batch_size=32)
    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {avg_loss:.4f}")

## Dataset Loading
Let's get the data directly from torchvision as we have seen during labs.

In [9]:
def get_data(data_dir="./data", transform=None):
    """Load Flowers102 train, validation and test sets.
    Args:
        data_dir (str): Directory where the dataset will be stored.
        transform (torch.Compose)
    Returns:
        tuple: A tuple containing the train, validation, and test sets.
    """
    train = torchvision.datasets.Flowers102(root=data_dir, split="train", download=True, transform=transform)
    val = torchvision.datasets.Flowers102(root=data_dir, split="val", download=True, transform=transform)
    test = torchvision.datasets.Flowers102(root=data_dir, split="test", download=True, transform=transform)
    return train, val, test

## Base and Novel categories
To split in base and novel categories we list all dataset classes, and count their number (we already know it's 102 but let's do it properly).
Then, we just allocate the first half to base categories and the remaining half to novel ones.
We can do this because we are simulating a real world application, but keep in mind this will not happen out there!

In [10]:
def base_novel_categories(dataset):
    # set returns the unique set of all dataset classes
    all_classes = set(dataset._labels)
    # and let's count them
    num_classes = len(all_classes)

    # here list(range(num_classes)) returns a list from 0 to num_classes - 1
    # then we slice the list in half and generate base and novel category lists
    base_classes = list(range(num_classes))[:num_classes//2]
    novel_classes = list(range(num_classes))[num_classes//2:]
    return base_classes, novel_classes

## Inspect Classes
Let's now visualize which are the base and novel classes.
To do so, we first get a dummy test set (without augmentations) as we are just interested in the dataset labels. Then, we split it useing `base_novel_categories`.
Finally, we use the hard-coded CLASS_NAMES to print the class in natural language.

> Note: the list of class names was only recently added to `torchvision.datasets.Flowers102`. To avoid useless errors that can occour to you, we decided to also provide such a list.

In [ ]:
_, _, tmp_test = get_data()
base_classes, novel_classes = base_novel_categories(tmp_test)
CLASS_NAMES = ["pink primrose", "hard-leaved pocket orchid", "canterbury bells", "sweet pea", "english marigold", "tiger lily", "moon orchid", "bird of paradise", "monkshood", "globe thistle", "snapdragon", "colt's foot", "king protea", "spear thistle", "yellow iris", "globe-flower", "purple coneflower", "peruvian lily", "balloon flower", "giant white arum lily", "fire lily", "pincushion flower", "fritillary", "red ginger", "grape hyacinth", "corn poppy", "prince of wales feathers", "stemless gentian", "artichoke", "sweet william", "carnation", "garden phlox", "love in the mist", "mexican aster", "alpine sea holly", "ruby-lipped cattleya", "cape flower", "great masterwort", "siam tulip", "lenten rose", "barbeton daisy", "daffodil", "sword lily", "poinsettia", "bolero deep blue", "wallflower", "marigold", "buttercup", "oxeye daisy", "common dandelion", "petunia", "wild pansy", "primula", "sunflower", "pelargonium", "bishop of llandaff", "gaura", "geranium", "orange dahlia", "pink-yellow dahlia?", "cautleya spicata", "japanese anemone", "black-eyed susan", "silverbush", "californian poppy", "osteospermum", "spring crocus", "bearded iris", "windflower", "tree poppy", "gazania", "azalea", "water lily", "rose", "thorn apple", "morning glory", "passion flower", "lotus", "toad lily", "anthurium", "frangipani", "clematis", "hibiscus", "columbine", "desert-rose", "tree mallow", "magnolia", "cyclamen", "watercress", "canna lily", "hippeastrum", "bee balm", "ball moss", "foxglove", "bougainvillea", "camellia", "mallow", "mexican petunia", "bromelia", "blanket flower", "trumpet creeper", "blackberry lily"]

""" Uncomment to see class names for base and novel categories"""
#print("Base Class Names:", [(i, CLASS_NAMES[i]) for i in base_classes])
#print("Novel Class Names:", [(i, CLASS_NAMES[i]) for i in novel_classes])

100%|██████████| 345M/345M [00:16<00:00, 20.4MB/s] 
100%|██████████| 502/502 [00:00<00:00, 1.32MB/s]
100%|██████████| 15.0k/15.0k [00:00<00:00, 21.9MB/s]


Base Class Names: [(0, 'pink primrose'), (1, 'hard-leaved pocket orchid'), (2, 'canterbury bells'), (3, 'sweet pea'), (4, 'english marigold'), (5, 'tiger lily'), (6, 'moon orchid'), (7, 'bird of paradise'), (8, 'monkshood'), (9, 'globe thistle'), (10, 'snapdragon'), (11, "colt's foot"), (12, 'king protea'), (13, 'spear thistle'), (14, 'yellow iris'), (15, 'globe-flower'), (16, 'purple coneflower'), (17, 'peruvian lily'), (18, 'balloon flower'), (19, 'giant white arum lily'), (20, 'fire lily'), (21, 'pincushion flower'), (22, 'fritillary'), (23, 'red ginger'), (24, 'grape hyacinth'), (25, 'corn poppy'), (26, 'prince of wales feathers'), (27, 'stemless gentian'), (28, 'artichoke'), (29, 'sweet william'), (30, 'carnation'), (31, 'garden phlox'), (32, 'love in the mist'), (33, 'mexican aster'), (34, 'alpine sea holly'), (35, 'ruby-lipped cattleya'), (36, 'cape flower'), (37, 'great masterwort'), (38, 'siam tulip'), (39, 'lenten rose'), (40, 'barbeton daisy'), (41, 'daffodil'), (42, 'sword 

## Split Dataset
The next step is to actually split the dataset into the base and novel categories we extract from `base_novel_categories`.
To split the data we need the dataset (obviously) and the list of base classes. If the sample label is not part of the base categories, then it must be part of the novel ones.

In [12]:
def split_data(dataset, base_classes):
    # these two lists will store the sample indexes
    base_categories_samples = []
    novel_categories_samples = []

    # we create a set of base classes to compute the test below in O(1)
    # this is optional and can be removed
    base_set = set(base_classes)

    # here we iterate over sample labels and also get the correspondent sample index
    for sample_id, label in enumerate(dataset._labels):
        if label in base_set:
            base_categories_samples.append(sample_id)
        else:
            novel_categories_samples.append(sample_id)

    # here we create the dataset subsets
    # the torch Subset is just a wrapper around the dataset
    # it simply stores the subset indexes and the original dataset (your_subset.dataset)
    # when asking for sample i in the subset, torch will look for its original position in the dataset and retrieve it
    # https://pytorch.org/docs/stable/data.html#torch.utils.data.Subset
    base_dataset = torch.utils.data.Subset(dataset, base_categories_samples)
    novel_dataset = torch.utils.data.Subset(dataset, novel_categories_samples)
    return base_dataset, novel_dataset

## Extract k shots
As the dataset already provides 10 train and validation shots, we do not need to extract them.
Beaware that Few-Shot Adaptation papers must do this operation as most datasets count significantly more samples in both the training and validation sets.

## Load CLIP

In [13]:
device = "cuda" if torch.cuda.is_available() else "cpu"
# available models = ['RN50', 'RN101', 'RN50x4', 'RN50x16', 'RN50x64', 'ViT-B/32', 'ViT-B/16', 'ViT-L/14', 'ViT-L/14@336px']
model, preprocess = clip.load("ViT-B/16", device=device)

# preprocess contains CLIP's pre-defined augmentations, let's inspect them!
preprocess

100%|███████████████████████████████████████| 335M/335M [00:04<00:00, 77.3MiB/s]


Compose(
    Resize(size=224, interpolation=bicubic, max_size=None, antialias=True)
    CenterCrop(size=(224, 224))
    <function _convert_image_to_rgb at 0x78461c920ea0>
    ToTensor()
    Normalize(mean=(0.48145466, 0.4578275, 0.40821073), std=(0.26862954, 0.26130258, 0.27577711))
)

## Load and Prepare Data
Here we get the three dataset split and pass clip pre-defined augmentations.
Then, we compute base and novel categories (in this case is redundand as we already did it before).
Finally, se split the three datasets into base and novel categories.
As we want to use the novel categories only for the test set, we drop `train_novel` and `val_novel`.

In [14]:
# get the three datasets
train_set, val_set, test_set = get_data(transform=preprocess)

# split classes into base and novel
base_classes, novel_classes = base_novel_categories(train_set)

# split the three datasets
train_base, _ = split_data(train_set, base_classes)
val_base, _ = split_data(val_set, base_classes)
test_base, test_novel = split_data(test_set, base_classes)

## Compute Zero-Shot Predictions

In [15]:
@torch.no_grad() # we don't want gradients
def eval(model, dataset, categories, batch_size, device, label=""):
    # let's set the model in evaluation mode
    model.eval()

    # Remap labels into a contiguous set starting from zero
    contig_cat2idx = {cat: idx for idx, cat in enumerate(categories)}

    # here we apply the standard CLIP template used for oxford flowers to all categories
    # and immediately tokenize each sentence (convert natural language into numbers - feel free to print the text input to inspect them)
    text_inputs = clip.tokenize(
        [f"a photo of a {CLASS_NAMES[c]}, a type of flower." for c in categories]
    ).to(device)

    # we can encode the text features once as they are shared for all images
    # therefore we do it outside the evaluation loop
    text_features = model.encode_text(text_inputs)
    # and here we normalize them (standard pratice with CLIP)
    text_features /= text_features.norm(dim=-1, keepdim=True)

    # simple dataloader creation
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=2)

    # here we store the number of correct predictions we will make
    correct_predictions = 0
    for image, target in tqdm(dataloader, desc=label):
        # base categories range from 0 to 50, whil novel ones from 51 to 101
        # therefore we must map categories to the [0, 50], otherwise we will have wrong predictions
        # Map targets in contiguous set starting from zero
        # Labels needs to be .long() in pytorch
        target = torch.Tensor([contig_cat2idx[t.item()] for t in target]).long()

        image = image.to(device)
        target = target.to(device)

        # forward image through CLIP image encoder
        image_features = model.encode_image(image)
        # and normalize
        image_features /= image_features.norm(dim=-1, keepdim=True)

        # here cosine similarity between image and text features and keep the argmax for every row (every image)
        predicted_class = (image_features @ text_features.T).argmax(dim=-1)
        # now we check which are correct, and sum them (False == 0, True == 1)
        correct_predictions += (predicted_class == target).sum().item()

    # and now we compute the accuracy
    accuracy = correct_predictions / len(dataset)
    return accuracy

base_accuracy = eval(model=model, dataset=test_base, categories=base_classes, batch_size=128, device=device, label="🧠 Zero-shot evaluation on Base Classes")
novel_accuracy = eval(model=model, dataset=test_novel, categories=novel_classes, batch_size=128, device=device, label="🧠 Zero-shot evaluation on Novel Classes")

print()
print(f"🔍 Base classes accuracy: {base_accuracy*100:.2f}%")
print(f"🔍 Novel classes accuracy: {novel_accuracy*100:.2f}%")


🧠 Zero-shot evaluation on Novel Classes: 100%|██████████| 29/29 [42:18<00:00, 87.55s/it]


🔍 Base classes accuracy: 71.29%
🔍 Novel classes accuracy: 78.24%


## Harmonic Mean
Few-Shot Adaptations papers usually report the Harmonic Mean.
The harmonic mean tends to mitigate the impact of large outliers (base accuracy) and aggravate the impact of small ones (novel accuracy).
Thus, achieving very high base accuracies at the expense of the novel accuracy will be penalized by the HM.

In [16]:
def harmonic_mean(base_accuracy, novel_accuracy):
    numerator = 2
    denominator = 1 / base_accuracy + 1 / novel_accuracy
    hm = numerator / denominator
    return hm

print(f"🔍 Harmonic Mean: {harmonic_mean(base_accuracy, novel_accuracy)*100:.2f}%")

🔍 Harmonic Mean: 74.60%
